# Super-Resolution Exercises

## Comparison of basic Super-resolution methods for SAR Ship classification

**Objective:** Implement and compare basic upsampling techniques using opencv.

**Tasks:**
1. Loading the dataset.
2. Downsampling each image.
3. Apply various upsampling techniques.
4. Comparing the scores and writing average scores.
5. Upscaling the images and storing them in a new folder for cnn's.

**Imports**

In [30]:
! pip install torchsr

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [31]:
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import numpy as np
import torch
# import matplotlib.pyplot as plt
from PIL import Image
# import cv2
import os
import warnings
from collections import defaultdict
from torchsr.models import edsr, rcan, carn

warnings.filterwarnings("ignore")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


**Code**

In [14]:
# def upsample(image, scale_factor, method):
#     if   method == "BILINEAR": method_up = Image.BILINEAR
#     elif method == "BICUBIC":  method_up = Image.BICUBIC
#     elif method == "NEAREST":  method_up = Image.NEAREST
#     elif method == "LANCZOS":  method_up = Image.LANCZOS
#     else:
#         print(method, "not found, using bicubic")
#         method_up = Image.BICUBIC

#     # Resize the image using bicubic interpolation
#     return image.resize(
#         (int(image.width * scale_factor), int(image.height * scale_factor)),
#         method_up
#     )

# main_path = "data/"

# polarizations = ["vv", "vh"]
# classes = ["Cargo", "Fishing", "Tanker"]

# scores = {}

# for pol in polarizations:
#     if pol not in scores:
#         scores[pol] = {}
#     for cl in classes:
#         if cl not in scores[pol]:
#             scores[pol][cl] = []
#         p_path = main_path + pol + "/" + cl
#         images_list = os.listdir(p_path)
#         t_images = len(images_list)
#         score_ssim_2x = [0, 0, 0, 0]
#         score_psnr_2x = [0, 0, 0, 0]
#         score_ssim_4x = [0, 0, 0, 0]
#         score_psnr_4x = [0, 0, 0, 0]
#         for i_image in images_list:
#             image_path = os.path.join(p_path, i_image)

#             image_hr = Image.open(image_path).convert('RGB')
#             # Get the original dimensions
#             original_width, original_height = image_hr.size

#             # Reduce dimensions by a factor of 2
#             new_width_2x = original_width // 2
#             new_height_2x = original_height // 2

#             image_lr_2 = image_hr.resize((new_width_2x, new_height_2x), Image.ANTIALIAS)
#             # image_2x.save("image_reduced_by_2x.jpg")  # Save the image resized by a factor of 2

#             # Reduce dimensions by a factor of 4
#             new_width_4x = original_width // 4
#             new_height_4x = original_height // 4

#             image_lr_4 = image_hr.resize((new_width_4x, new_height_4x), Image.ANTIALIAS)

#             for idx, method in enumerate(["NEAREST", "BILINEAR", "BICUBIC", "LANCZOS"]):
#                 image_up_2 = upsample(image_lr_2, scale_factor=2, method=method)
#                 psnr_value_2 = psnr(np.array(image_hr), np.array(image_up_2))
#                 score_psnr_2x[idx] += psnr_value_2
#                 ssim_value_2 = ssim(np.array(image_hr), np.array(image_up_2), win_size=3, channel_axis=-1)
#                 score_ssim_2x[idx] += ssim_value_2

#                 image_up_4 = upsample(image_lr_4, scale_factor=4, method=method)
#                 psnr_value_4 = np.round(psnr(np.array(image_hr), np.array(image_up_4)), 2)
#                 score_psnr_4x[idx] += psnr_value_4
#                 ssim_value_4 = np.round(ssim(np.array(image_hr), np.array(image_up_4), win_size=3, channel_axis=-1), 4)
#                 score_ssim_4x[idx] += ssim_value_4
        
#         scores[pol][cl].append(list(numpy.array(score_ssim_2x)/t_images))
#         scores[pol][cl].append(list(numpy.array(score_psnr_2x)/t_images))
#         scores[pol][cl].append(list(numpy.array(score_ssim_4x)/t_images))
#         scores[pol][cl].append(list(numpy.array(score_psnr_4x)/t_images))

In [ ]:
def upsample(image, target_size, method):
    method_map = {
        "BILINEAR": Image.BILINEAR,
        "BICUBIC": Image.BICUBIC,
        "NEAREST": Image.NEAREST,
        "LANCZOS": Image.LANCZOS
    }
    method_up = method_map.get(method, Image.BICUBIC)
    return image.resize(target_size, method_up)


main_path = "data/"
polarizations = ["vv", "vh"]
classes = ["Cargo", "Fishing", "Tanker"]

scores = defaultdict(lambda: defaultdict(dict))

for pol in polarizations:
    print(pol)
    for cl in classes:
        print(cl)
        p_path = os.path.join(main_path, pol, cl)
        images_list = os.listdir(p_path)
        t_images = len(images_list)

        score_ssim_2x = [0, 0, 0, 0]
        score_psnr_2x = [0, 0, 0, 0]
        score_ssim_4x = [0, 0, 0, 0]
        score_psnr_4x = [0, 0, 0, 0]

        for i_image in images_list:
            image_path = os.path.join(p_path, i_image)

            try:
                image_hr = Image.open(image_path).convert('RGB')
            except Exception as e:
                print(f"Error opening image {image_path}: {e}")
                continue


            original_width, original_height = image_hr.size
            
            if original_width < 2 or original_height < 2:
                print("Image too small to downscale:", image_path)
                continue
            image_lr_2 = image_hr.resize((original_width // 2, original_height // 2), Image.Resampling.LANCZOS)
            image_lr_4 = image_hr.resize((original_width // 4, original_height // 4), Image.Resampling.LANCZOS)
            image_hr_np = np.array(image_hr)

            for idx, method in enumerate(["NEAREST", "BILINEAR", "BICUBIC", "LANCZOS"]):
                # image_up_2 = upsample(image_lr_2, scale_factor=2, method=method)
                image_up_2 = upsample(image_lr_2, (original_width, original_height), method=method)
                image_up_2_np = np.array(image_up_2)

                psnr_value_2 = np.round(psnr(image_hr_np, image_up_2_np), 2)
                score_psnr_2x[idx] += psnr_value_2

                ssim_value_2 = np.round(ssim(image_hr_np, image_up_2_np, win_size=3, channel_axis=-1), 4)
                score_ssim_2x[idx] += ssim_value_2

                # image_up_4 = upsample(image_lr_4, scale_factor=4, method=method)
                image_up_4 = upsample(image_lr_4, (original_width, original_height), method=method)
                image_up_4_np = np.array(image_up_4)

                psnr_value_4 = np.round(psnr(image_hr_np, image_up_4_np), 2)
                score_psnr_4x[idx] += psnr_value_4

                ssim_value_4 = np.round(ssim(image_hr_np, image_up_4_np, win_size=3, channel_axis=-1), 4)
                score_ssim_4x[idx] += ssim_value_4
        
        # Normalize the scores first to avoid redundant np.array() and division operations
        ssim_2x_normalized = list(np.array(score_ssim_2x) / t_images)
        psnr_2x_normalized = list(np.array(score_psnr_2x) / t_images)
        ssim_4x_normalized = list(np.array(score_ssim_4x) / t_images)
        psnr_4x_normalized = list(np.array(score_psnr_4x) / t_images)

        # Assign the values to the respective dictionary keys
        scores[pol][cl]['ssim_2x'] = ssim_2x_normalized
        scores[pol][cl]['psnr_2x'] = psnr_2x_normalized
        scores[pol][cl]['ssim_4x'] = ssim_4x_normalized
        scores[pol][cl]['psnr_4x'] = psnr_4x_normalized


In [29]:
%timeit
def upsample(image, scale_factor, method):
    if   method == "BILINEAR": method_up = Image.BILINEAR
    elif method == "BICUBIC":  method_up = Image.BICUBIC
    elif method == "NEAREST":  method_up = Image.NEAREST
    elif method == "LANCZOS":  method_up = Image.LANCZOS
    else:
        print(method, "not found, using bicubic")
        method_up = Image.BICUBIC

    # Resize the image using bicubic interpolation
    return image.resize(
        (int(image.width * scale_factor), int(image.height * scale_factor)),
        method_up
    )


main_path = "data/"
storing_data = "high_data/"
polarizations = ["vv", "vh"]
classes = ["Cargo", "Fishing", "Tanker"]


for cl in classes:
    print(cl)
    vv_path = os.path.join(main_path, polarizations[0], cl)
    vh_path = os.path.join(main_path, polarizations[1], cl)
    images_list = os.listdir(vv_path)

    for i_image in images_list:
        vv_image_path = os.path.join(vv_path, i_image)
        vh_image_path = os.path.join(vh_path, i_image)

        try:
            vv_image_hr = Image.open(vv_image_path).convert('RGB')
            vh_image_hr = Image.open(vh_image_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {vv_image_path}: {e}")
            continue


        original_width, original_height = vv_image_hr.size
        
        if original_width < 2 or original_height < 2:
            print("Image too small to downscale:", image_path)
            continue
        
        for idx, method in enumerate(["NEAREST", "BILINEAR", "BICUBIC", "LANCZOS"]):
            vv_image_up_2 = upsample(vv_image_hr, scale_factor=2, method=method)
            vh_image_up_2 = upsample(vh_image_hr, scale_factor=2, method=method)

            hr_path = os.path.join(storing_data, method, "all_sar", cl)
            
            os.makedirs(hr_path, exist_ok=True)
            vv_image_up_2.save(os.path.join(hr_path, i_image.replace(".tif", "_vv.tif")))
            vh_image_up_2.save(os.path.join(hr_path, i_image.replace(".tif", "_vh.tif")))

Cargo
Image too small to downscale: data/vh/Tanker/Tanker7074.tif
Fishing
Tanker


In [8]:
import pickle

# Convert the defaultdict to a regular dict
scores_dict = {pol: {cl: dict(values) for cl, values in classes.items()} for pol, classes in scores.items()}

# Save the scores dictionary to a file
with open('super_resolution_scores_dict.pkl', 'wb') as file:
    pickle.dump(scores_dict, file)


In [9]:
# Load the scores dictionary from a file
with open('super_resolution_scores_dict.pkl', 'rb') as file:
    loaded_scores = pickle.load(file)


In [17]:
! pip install pandas openpyxl


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [21]:
for pol, classes in scores.items():
    print(pol, classes)
    for cl, metrics in classes.items():
        print(cl, metrics)

vv defaultdict(<class 'dict'>, {'Cargo': {'ssim_2x': [0.8932723552706013, 0.9215749952856858, 0.9271920799547377, 0.928380482745617], 'psnr_2x': [31.08239487082778, 35.17101074863282, 36.54377522157269, 37.33758815764645], 'ssim_4x': [0.8204931359607761, 0.849147708844051, 0.8465192721101246, 0.8406072034697365], 'psnr_4x': [28.31146520837261, 29.567656043748844, 30.083909108052083, 30.340969262681426]}, 'Fishing': {'ssim_2x': [0.6910172661870506, 0.7606539568345323, 0.7810733812949641, 0.7877597122302157], 'psnr_2x': [24.282661870503592, 27.624892086330938, 28.839280575539565, 29.518561151079137], 'ssim_4x': [0.5058107913669063, 0.5506100719424457, 0.5496856115107913, 0.5428654676258994], 'psnr_4x': [21.73410071942447, 22.438057553956835, 22.805899280575538, 22.980647482014387]}, 'Tanker': {'ssim_2x': [0.8726104109589039, 0.9068726575342483, 0.9132987945205476, 0.9145852054794511], 'psnr_2x': [29.686641095890305, 33.80355616438357, 35.2623616438356, 36.11446575342467], 'ssim_4x': [0.7

In [22]:
import pandas as pd

# Convert the scores dictionary to a list for Excel
data = []

for pol, classes in scores.items():
    for cl, metrics in classes.items():
        # Create a flat dictionary for each class
        row = {
            'Polarization': pol,
            'Class': cl,
            'SSIM_2x': metrics.get('ssim_2x', []),
            'PSNR_2x': metrics.get('psnr_2x', []),
            'SSIM_4x': metrics.get('ssim_4x', []),
            'PSNR_4x': metrics.get('psnr_4x', [])
        }
        # Add the row to the data list
        data.append(row)

# Create a DataFrame
df = pd.DataFrame(data)

# Save to Excel
df.to_excel('super_resolution_scores.xlsx', index=False)


In [23]:
import pandas as pd

# Define the method labels
method_labels = ["NEAREST", "BILINEAR", "BICUBIC", "LANCZOS"]

# Convert the scores dictionary to a list for Excel
data = []

for pol, classes in scores.items():
    for cl, metrics in classes.items():
        # Create a flat dictionary for each class
        row = {
            'Polarization': pol,
            'Class': cl,
        }
        # Add rounded scores for each method
        for idx, method in enumerate(method_labels):
            row[f'SSIM_2x_{method}'] = round(metrics.get('ssim_2x', [0])[idx], 4) if idx < len(metrics.get('ssim_2x', [])) else None
            row[f'PSNR_2x_{method}'] = round(metrics.get('psnr_2x', [0])[idx], 2) if idx < len(metrics.get('psnr_2x', [])) else None
            row[f'SSIM_4x_{method}'] = round(metrics.get('ssim_4x', [0])[idx], 4) if idx < len(metrics.get('ssim_4x', [])) else None
            row[f'PSNR_4x_{method}'] = round(metrics.get('psnr_4x', [0])[idx], 2) if idx < len(metrics.get('psnr_4x', [])) else None
            
        # Add the row to the data list
        data.append(row)

# Create a DataFrame
df = pd.DataFrame(data)

# Save to Excel
df.to_excel('super_resolution_scores.xlsx', index=False)


# SR using deep learning

In [ ]:
# List of models to iterate through
models = {
    'EDSR': [edsr(scale=2, pretrained=True), edsr(scale=4, pretrained=True)],
    'RCAN': [rcan(scale=2, pretrained=True), rcan(scale=4, pretrained=True)],
    'CARN': [carn(scale=2, pretrained=True), carn(scale=4, pretrained=True)]
}

# Move models to the device once, and freeze them
for idx, (model_name, model) in enumerate(models.items()):
    model[0].to(device).eval()  # Move 2x model to device
    model[1].to(device).eval()  # Move 4x model to device
    for param in model[0].parameters():
        param.requires_grad = False
    for param in model[1].parameters():
        param.requires_grad = False

main_path = "data/"
polarizations = ["vv", "vh"]
classes = ["Cargo", "Fishing", "Tanker"]

scores = defaultdict(lambda: defaultdict(dict))

for pol in polarizations:
    print(pol)
    for cl in classes:
        print(cl)
        p_path = os.path.join(main_path, pol, cl)
        images_list = os.listdir(p_path)
        t_images = len(images_list)

        score_ssim_2x = [0, 0, 0]
        score_psnr_2x = [0, 0, 0]
        score_ssim_4x = [0, 0, 0]
        score_psnr_4x = [0, 0, 0]
        for idx, (model_name, model) in enumerate(models.items()):
            model[0].to(device)  # Move models to device once, outside of image loop
            model[1].to(device)
            model[0].eval()
            model[1].eval()
            for i_image in images_list:
                image_path = os.path.join(p_path, i_image)

                try:
                    image_hr = Image.open(image_path).convert('RGB')
                except Exception as e:
                    print(f"Error opening image {image_path}: {e}")
                    continue


                original_width, original_height = image_hr.size
                
                if original_width < 2 or original_height < 2:
                    print("Image too small to downscale:", image_path)
                    continue
                
                image_lr_2 = image_hr.resize((original_width // 2, original_height // 2), Image.Resampling.LANCZOS)
                lr_tensor_2 = ToTensor()(image_lr_2).unsqueeze(0)
                image_lr_4 = image_hr.resize((original_width // 4, original_height // 4), Image.Resampling.LANCZOS)
                lr_tensor_4 = ToTensor()(image_lr_4).unsqueeze(0)
                image_hr_np = np.array(image_hr)
                
                # Inside the image processing loop
                with torch.no_grad():
                    sr_tensor2 = model[0](lr_tensor_2.to(device))
                    sr_tensor4 = model[1](lr_tensor_4.to(device))

                sr_image2 = ToPILImage()(sr_tensor2.squeeze())
                sr_image4 = ToPILImage()(sr_tensor4.squeeze())

                sr_image2_np = np.array(sr_image2)
                sr_image4_np = np.array(sr_image4)

                psnr_value_2 = np.round(psnr(image_hr_np, sr_image2_np), 2)
                score_psnr_2x[idx] += psnr_value_2

                ssim_value_2 = np.round(ssim(image_hr_np, sr_image2_np, win_size=3, channel_axis=-1), 4)
                score_ssim_2x[idx] += ssim_value_2

                psnr_value_4 = np.round(psnr(image_hr_np, sr_image4_np), 2)
                score_psnr_4x[idx] += psnr_value_4

                ssim_value_4 = np.round(ssim(image_hr_np, sr_image4_np, win_size=3, channel_axis=-1), 4)
                score_ssim_4x[idx] += ssim_value_4
        
        # Normalize the scores first to avoid redundant np.array() and division operations
        ssim_2x_normalized = list(np.array(score_ssim_2x) / t_images)
        psnr_2x_normalized = list(np.array(score_psnr_2x) / t_images)
        ssim_4x_normalized = list(np.array(score_ssim_4x) / t_images)
        psnr_4x_normalized = list(np.array(score_psnr_4x) / t_images)

        # Assign the values to the respective dictionary keys
        scores[pol][cl]['ssim_2x'] = ssim_2x_normalized
        scores[pol][cl]['psnr_2x'] = psnr_2x_normalized
        scores[pol][cl]['ssim_4x'] = ssim_4x_normalized
        scores[pol][cl]['psnr_4x'] = psnr_4x_normalized

In [ ]:
# import pickle
# from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
# import numpy as np
# import torch
# # import matplotlib.pyplot as plt
# from PIL import Image
# # import cv2
# import os
# import warnings
# from collections import defaultdict
# from torchsr.models import edsr, rcan, carn

# warnings.filterwarnings("ignore")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

# # List of models to iterate through
# models = {
#     'EDSR': [edsr(scale=2, pretrained=True), edsr(scale=4, pretrained=True)],
#     'RCAN': [rcan(scale=2, pretrained=True), rcan(scale=4, pretrained=True)],
#     'CARN': [carn(scale=2, pretrained=True), carn(scale=4, pretrained=True)]
# }


# # Move models to the device once, and freeze them
# for idx, (model_name, model) in enumerate(models.items()):
#     model[0].to(device).eval()  # Move 2x model to device
#     model[1].to(device).eval()  # Move 4x model to device
#     for param in model[0].parameters():
#         param.requires_grad = False
#     for param in model[1].parameters():
#         param.requires_grad = False

# main_path = "data/"
# polarizations = ["vv", "vh"]
# classes = ["Cargo", "Fishing", "Tanker"]

# scores = defaultdict(lambda: defaultdict(dict))

# # Custom Dataset for Image Loading
# class ImageDataset(Dataset):
#     def __init__(self, image_paths):
#         self.image_paths = image_paths
    
#     def __len__(self):
#         return len(self.image_paths)
    
#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         image = Image.open(image_path).convert('RGB')
#         return image_path, image

# # Optimized Transformations
# to_tensor = ToTensor()
# to_pil_image = ToPILImage()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# for pol in polarizations:
#     print(pol)
#     for cl in classes:
#         print(cl)
#         p_path = os.path.join(main_path, pol, cl)
#         images_list = os.listdir(p_path)
#         t_images = len(images_list)

#         # Create DataLoader for efficient image loading
#         image_dataset = ImageDataset([os.path.join(p_path, img) for img in images_list])
#         image_loader = DataLoader(image_dataset, batch_size=32, num_workers=4, shuffle=False)

#         score_ssim_2x = [0, 0, 0]
#         score_psnr_2x = [0, 0, 0]
#         score_ssim_4x = [0, 0, 0]
#         score_psnr_4x = [0, 0, 0]

#         for batch in image_loader:
#             image_paths, images_hr = batch
#             for image_path, image_hr in zip(image_paths, images_hr):
#                 original_width, original_height = image_hr.size
#                 if original_width < 2 or original_height < 2:
#                     print(f"Image too small to downscale: {image_path}")
#                     continue

#                 # Generate low-resolution images (2x and 4x downscaled)
#                 image_lr_2 = image_hr.resize((original_width // 2, original_height // 2), Image.LANCZOS)
#                 image_lr_4 = image_hr.resize((original_width // 4, original_height // 4), Image.LANCZOS)

#                 lr_tensor_2 = to_tensor(image_lr_2).unsqueeze(0).to(device)
#                 lr_tensor_4 = to_tensor(image_lr_4).unsqueeze(0).to(device)
#                 image_hr_np = np.array(image_hr)

#                 for idx, (model_name, model) in enumerate(models.items()):
#                     # Super-resolve images with both 2x and 4x models
#                     with torch.no_grad():
#                         sr_tensor_2 = model[0](lr_tensor_2).squeeze().cpu()  # Move output back to CPU
#                         sr_tensor_4 = model[1](lr_tensor_4).squeeze().cpu()

#                     # Convert tensor back to images
#                     sr_image_2_np = np.array(to_pil_image(sr_tensor_2))
#                     sr_image_4_np = np.array(to_pil_image(sr_tensor_4))

#                     # Compute PSNR and SSIM scores for both 2x and 4x upscaling
#                     psnr_value_2 = np.round(psnr(image_hr_np, sr_image_2_np), 2)
#                     ssim_value_2 = np.round(ssim(image_hr_np, sr_image_2_np, win_size=3, channel_axis=-1), 4)

#                     score_psnr_2x[idx] += psnr_value_2
#                     score_ssim_2x[idx] += ssim_value_2

#                     psnr_value_4 = np.round(psnr(image_hr_np, sr_image_4_np), 2)
#                     ssim_value_4 = np.round(ssim(image_hr_np, sr_image_4_np, win_size=3, channel_axis=-1), 4)

#                     score_psnr_4x[idx] += psnr_value_4
#                     score_ssim_4x[idx] += ssim_value_4

#         # Normalize the scores after processing all images
#         ssim_2x_normalized = list(np.array(score_ssim_2x) / t_images)
#         psnr_2x_normalized = list(np.array(score_psnr_2x) / t_images)
#         ssim_4x_normalized = list(np.array(score_ssim_4x) / t_images)
#         psnr_4x_normalized = list(np.array(score_psnr_4x) / t_images)

#         # Assign the values to the respective dictionary keys
#         scores[pol][cl]['ssim_2x'] = ssim_2x_normalized
#         scores[pol][cl]['psnr_2x'] = psnr_2x_normalized
#         scores[pol][cl]['ssim_4x'] = ssim_4x_normalized
#         scores[pol][cl]['psnr_4x'] = psnr_4x_normalized

# # Convert the defaultdict to a regular dict
# scores_dict = {pol: {cl: dict(values) for cl, values in classes.items()} for pol, classes in scores.items()}

# # Save the scores dictionary to a file
# with open('super_resolution_scores_dict.pkl', 'wb') as file:
#     pickle.dump(scores_dict, file)


In [35]:
# Load the scores dictionary from a file
with open('super_resolution_scores_dict.pkl', 'rb') as file:
    loaded_scores = pickle.load(file)

In [38]:
import pandas as pd

# Define the method labels
method_labels = ["EDSR", "RCAN", "CARN"]

# Convert the scores dictionary to a list for Excel
data = []

for pol, classes in loaded_scores.items():
    for cl, metrics in classes.items():
        # Create a flat dictionary for each class
        row = {
            'Polarization': pol,
            'Class': cl,
        }
        # Add rounded scores for each method
        for idx, method in enumerate(method_labels):
            row[f'SSIM_2x_{method}'] = round(metrics.get('ssim_2x', [0])[idx], 4) if idx < len(metrics.get('ssim_2x', [])) else None
            row[f'PSNR_2x_{method}'] = round(metrics.get('psnr_2x', [0])[idx], 2) if idx < len(metrics.get('psnr_2x', [])) else None
            row[f'SSIM_4x_{method}'] = round(metrics.get('ssim_4x', [0])[idx], 4) if idx < len(metrics.get('ssim_4x', [])) else None
            row[f'PSNR_4x_{method}'] = round(metrics.get('psnr_4x', [0])[idx], 2) if idx < len(metrics.get('psnr_4x', [])) else None
            
        # Add the row to the data list
        data.append(row)

# Create a DataFrame
df = pd.DataFrame(data)

# Save to Excel
df.to_excel('super_resolution_scores_dl.xlsx', index=False)
